Note that the labelling of these sections is based on the version of the paper completed on 15/08/22. This cell should be updated in the event of a change to the labelling in the paper.

Currently running on Sagemath version 9.4.rc0, Release Date: 2021-07-27.
Using Python 3.9.5.

This notebook is intended to be ran sequentially from the beginning. Changing the order in which cells are ran may lead to errors. 

## Theorem 5.9

In [1]:
# We will first use the method of Kallel and Sjerve

from sage.geometry.hyperplane_arrangement.affine_subspace import AffineSubspace
from sage.groups.perm_gps.partn_ref.refinement_graphs import get_orbits

# Initialise the Curve
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface
R.<x, y> = QQ[]
f = x*(y^5+1) + (x*y)^2 - x^4*y - 2*y^3
integration_method = 'rigorous'
prec = 60
S = RiemannSurface(f, prec=prec, integration_method=integration_method)
Mlist = S.symplectic_isomorphisms()
g = S.genus

# Initialise some useful objects
I2g = matrix.identity(2*g)
Ig = matrix.identity(g)
Jg = block_matrix([[matrix.zero(g), Ig], [-Ig, matrix.zero(g)]])
F2 = FiniteField(2)
Vbars = [vector([sum([A[j,i]*A[k,i]*Jg[j,k] for j in range(2*g) for k in range(j, 2*g)]) 
                 for i in range(2*g)], F2) for A in Mlist] 
Mbars = [matrix(m, ring=F2) for m in Mlist]
# XS will be the vectors corresponding to theta characteristics
XS = [vector(Integer(n).digits(2, padto=2*g), F2) for n in range(4^g)]
# split by parity (as to include odd characteristics here)
odds = [X for X in XS if sum([X[i]*X[i+g] for i in range(g)])]
evens = [X for X in XS if not sum([X[i]*X[i+g] for i in range(g)])]
print("# odd spin structures:", 2^(g-1)*(2^g-1))
print("# even spin structures:", 2^(g-1)*(2^g+1),"\n")

# Use get orbits to have the orbit decomposition.
print("Partitions:")
for parity in [odds, evens]:
    SS_Perms_exact = [[parity.index(A.transpose()*xi+V) for xi in parity] for A, V in zip(Mbars, Vbars)]
    SS_orbits_exact = get_orbits(SS_Perms_exact, len(parity))
    lengths = [len(o) for o in SS_orbits_exact]
    print(sum(lengths),"=",lengths)

print()
# We can also intersect the corresponding affine subspaces. 
MIs = [M-I2g for M in Mbars]
X0s = [MI.solve_left(-V) for MI, V in zip(MIs, Vbars)]
Kers = [MI.kernel() for MI in MIs]
AS = [AffineSubspace(X0, K) for X0, K in zip(X0s, Kers)]
Sols = AS[0]
for AI in AS[1:]:
    Sols = Sols.intersection(AI)
print("Space of invariant spin structures:", Sols)

# odd spin structures: 120
# even spin structures: 136 

Partitions:
120 = [20, 20, 60, 20]
136 = [30, 10, 30, 30, 5, 5, 10, 10, 5, 1]

Space of invariant spin structures: Affine space p + W where:
  p = (0, 0, 0, 0, 1, 0, 0, 1)
  W = Vector space of degree 8 and dimension 0 over Finite Field of size 2
Basis matrix:
[]


In [7]:
# To do the calculation we need some legwork completed in the automorphism group notebook,
# namely to calculate the automorphism group in the HC model as a group of substitutions. 
# See that notebook for more of a discussion of this first bit of code. 

# Because we will want this Riemann surface to have the AJ map, we will redefine. 
load("riemann_surface.py")
R.<x, y> = QQ[]
f = x*(y^5+1) + (x*y)^2 - x^4*y - 2*y^3
integration_method = 'rigorous'
prec = 60
S = RiemannSurface(f, prec=prec, integration_method=integration_method)
Mlist = S.symplectic_isomorphisms()

#####
K.<z> = CyclotomicField(5)
Tnum = S.tangent_representation_numerical(Mlist)
zc = S._CC(z)
indeps = [zc^m for m in range(z.multiplicative_order()-1)]
def lincomb(v):
    ls = [v]+indeps
    w = vector(K, pari.lindep(ls).sage()) 
    if w[0] != 0 :
        return K(list(-w[1:]/w[0]))    
Talg = [matrix(K, g, g, [lincomb(a) for a in tnum.list()]) for tnum in Tnum]
L = PolynomialRing(K, names=['x','y'])
vB = vector([L(b) for b in S.cohomology_basis()]) 
def mat_to_subs(M):
    w = M*vB
    yp = w[3]/w[0]
    xp = (w[1]+yp*w[3])/w[2]
    return (xp, yp)
auts_as_subs = [mat_to_subs(talg) for talg in Talg]

half_len = len(auts_as_subs)/2
if all([auts_as_subs.index(auts_as_subs[-i])==half_len-i for i in range(1, half_len)]):
    print("removing redundancy")
    auts_as_subs = auts_as_subs[:half_len]   
####
# We calculate the AJ map of a certain theta characteristic as done in the Weierstass notebook
# Now compute the canonical divisor (using a known theta characteristic)
F = Curve(f).function_field()
divx = F(x).divisor()
c, d, b, a = sorted(divx.support(), key = lambda si: divx.valuation(si))
avoid = [a, b, c, d]
# a_avoid, _ = S.strong_approximation(1*a, avoid)
# a_list = S.divisor_to_divisor_list(a_avoid)
# AJ_bp_a = S.abel_jacobi(a_list)
Delta = a+3*b-d
Delta_avoid, _ = S.strong_approximation(Delta, avoid)
Delta_list = S.divisor_to_divisor_list(Delta_avoid)
AJ_bp_D = S.abel_jacobi(Delta_list)
####

# We store the point that is the basepoint of the Abel-Jacobi map 
bpCC = (S._vertices[S._basepoint[0]], S._wvalues[S._basepoint[0]][S._basepoint[1]])
# We will calculate those elements of Tnum that are indeed the pullback of an automorphism
# on the differentials, and not this pullback multiplied -1 in the Jacobian. These are 
# equivalently the orientation preserving operations. 
Tnum_op = []

# We expect the output of the symplectic isomorphisms to be s.t. T[i]+T[120+i]==0, 
# and we rely on this for the next calculation, so we check it. 
if not all([Talg[i]+Talg[120+i]==0 for i in range(120)]):
    raise ValueError("Matrices are not distributed as expected")

# We pick out the correct matrices in Tnum as described below
for sigma, L1, L2 in zip(auts_as_subs, Tnum[:120], Tnum[120:]):
    sigma_bp = tuple(sigmaj(*bpCC) for sigmaj in sigma)
    tA1 = (L1-Ig)*2*AJ_bp_D+2*(g-1)*S._aj_based(sigma_bp)
    tA2 = (L2-Ig)*2*AJ_bp_D+2*(g-1)*S._aj_based(sigma_bp)
    if S.reduce_over_period_lattice(tA1).norm()<1e-10:
        Tnum_op.append(L1)
    elif S.reduce_over_period_lattice(tA2).norm()<1e-10:
        Tnum_op.append(L2)
    else:
        raise ValueError("neither transform is orientation preserving")

if (len(Tnum_op)!=120):
    raise ValueError("Incorrect number of automorphisms found.")

    
# As shown below we need to only calculate the transform of the half-lattice vectors. 
V = VectorSpace(S._CC, 2*g)
XS = [V(vector(Integer(n).digits(2, padto=2*g), F2)) for n in range(4^g)]
PM  = S.period_matrix()
half_lattices = [PM*xi/2 for xi in XS]

# The following function will give a way to find which half lattice one is transformed to 
def index_of_transformed_hl(hl, L):
    hl2 = L*hl
    ds = [S.reduce_over_period_lattice(hl2-hls).norm() for hls in half_lattices]
    # If using checks, insert a check clause here, for now we assume these have all worked
    return ds.index(min(ds))

# To generate the whole group we will just take the action of R, S, and U
# We need the matrices corresponding to R and S, and then their associated transform
K.<z> = CyclotomicField(5)
rs = (polygen(K)^2-5).roots(multiplicities=False)
s5 = CC(sqrt(5))
remainders = [(K.embeddings(CC)[0](r)-s5).abs() for r in rs]
s5zs = rs[remainders.index(min(remainders))]
# To avoid overlap of notation we will call them Rp and Sp
Sp = Matrix([[1,0,0],[0,z^(1),0],[0,0,z^(-1)]])
Rp = Matrix([[1,2,2],[1,z^2+z^(-2),z^1+z^(-1)],[1,z^1+z^(-1),z^2+z^(-2)]])/s5zs
# Now we have the matrices get the action. 
L = PolynomialRing(K, names=['x','y'])
RP_action = Rp*vector([L.gen(0), L.gen(1), 1])
RP_as_subs = (RP_action[0]/RP_action[2], RP_action[1]/RP_action[2])
SP_action = Sp*vector([L.gen(0), L.gen(1), 1])
SP_as_subs = (SP_action[0]/SP_action[2], SP_action[1]/SP_action[2])
# We get U as our simplest non-linear as before
non_lin = []
for i in range(120):
    xp, yp = auts_as_subs[i]
    xnum = xp.numerator()
    xden = xp.denominator()
    ynum = yp.numerator()
    yden = yp.denominator()
    nds = [xnum, xden, ynum, yden]
    if not all([ndi.degree()<=1 for ndi in nds]):
        non_lin.append(auts_as_subs[i])
if len(non_lin) != 60:
    raise ValueError    
lengths = [len(aut[0]._repr_())+len(aut[1]._repr_()) for aut in non_lin]
nl = non_lin[lengths.index(min(lengths))]

# With this we can collect the information
RSU_as_subs = [RP_as_subs, SP_as_subs, nl]
RSU_as_matrices = [Tnum_op[auts_as_subs.index(geni)] for geni in RSU_as_subs]

# We take note of the time we expect the cell will take to run to be responsible. 
import time
ct = time.time()
index_of_transformed_hl(half_lattices[0], RSU_as_matrices[0])
ct2 = time.time()
print('warning, next part of cell will\
 take approximately {} minutes to run'.format((3*len(half_lattices)*(ct2-ct))/60))

HL_Perms_num = [[index_of_transformed_hl(hli, Li) for hli in half_lattices] 
                for Li in RSU_as_matrices]
# This next line takes a while 
nSS = 4^g
HL_orbits_num = get_orbits(HL_Perms_num, nSS)
lengths = [len(o) for o in HL_orbits_num]
print(sum(lengths),"=",lengths)

256 = [1, 30, 10, 5, 30, 10, 20, 60, 5, 20, 20, 5, 10, 30]


It's worth writing down what is going on in the above calculation. We have 
$$
A_\ast(K) = \int_\ast^K {\omega}
$$
and so when we act on the left with $L$ the matrix corresponding to automorphism $\sigma$
$$
L A_\ast(K) = \int_\ast^K L \omega = \int_\ast^K \sigma^\ast \omega = \int_{\sigma(\ast)}^{\sigma(K)} \omega = \int_\ast^{\sigma(K)} \omega - (\deg K) \int_\ast^{\sigma(\ast)} \omega 
$$
and hence 
$$
(L-I)A_\ast(K)+2(g-1)A_\ast(\sigma(\ast)) = \int_K^{\sigma(K)} \omega
$$
Then $\sigma(K) \sim K$ iff the RHS is 0 in the Jacobian. As an automorphism must preserve the canonical divisor, this let's us pick out the correct matrices. 

Now given a spin structure $D$ s.t. $2D=K$ we should have $2A_\ast(D) = A_\ast(2D) = A_\ast(K)$ and to the AJ images of the spin structures can differ only by a half period $h = \Omega n$, that is $A_\ast(D) = A_\ast(\Delta)+h$. Now as spin structures are mapped to each other we expect $\sigma(D) = D^\prime$, and so 
$$
L A_\ast(D) = \int_{\sigma(\ast)}^{\sigma(D)} \omega = A_\ast(D^\prime) - (g-1) A_\ast(\sigma(\ast))
$$


As in the previous case we see 
$$
L [A_\ast(\Delta)+h] = [LA_\ast(\Delta)] + Lh = [A_\ast(\Delta) - (g-1) A_\ast(\sigma(\ast))] + Lh = A_\ast(\Delta) + [Lh - (g-1) A_\ast(\sigma(\ast))]
$$
Thus in order to find the transform of the spin structures we need to just look at the transforms of the half-lattice vectors.

## Theorem 5.11

In [8]:
# The above numerical calculation was certainly much more involved than that of KS06, and slower. 
# The main benefit is that it will let us identify whether a candidate theta characteristic is invariant
# directly, which KS06 will not. We demonstrate this by proving theorem 5.11
for sigma, Li in zip(RSU_as_subs, RSU_as_matrices):
    sigma_bp = tuple(sigmaj(*bpCC) for sigmaj in sigma)
    tA = (Li-Ig)*AJ_bp_D+(g-1)*S._aj_based(sigma_bp)
    print((S.reduce_over_period_lattice(tA).norm()<1e-10))

True
True
True


## Proposition 5.13

In [19]:
# We now want to verify that, in the RR homology basis, we have that the unique invariant theta 
# characteristic is the divisor of the Szego kernel. 
# This will require high-precision calculation, so we will repeat lots of the calculation above. 
from sage.schemes.riemann_surfaces.riemann_surface import numerical_inverse

# Initialise the curve 
R.<x, y> = QQ[]
f = x*(y^5+1) + (x*y)^2 - x^4*y - 2*y^3
integration_method = 'rigorous'
prec = 200
S = RiemannSurface(f, prec=prec, integration_method=integration_method)
g = S.genus

# Define our canonical divisor. 
K_can = F(y^3-x).divisor() + F(x).differential().divisor() - F(f.derivative(y)).divisor()
# We make some check to ensure a, b, c, d, have been assigned correctly
if K_can-3*a-2*c-1*d:
    raise ValueError("a,b,c,d have been assigned incorrectly")
    
# We recalculate the period matrix and normalise, as this is required for the RCV
PM = S.period_matrix()
AM = PM[:,0:g]
AInv = numerical_inverse(AM)
PM_normalised = AInv*PM

# We can calculate all possible half-periods in the normalised lattice
V = VectorSpace(S._CC, 2*g)
XS = [V(vector(Integer(n).digits(2, padto=2*g), F2)) for n in range(4^g)]
half_lattices_normalised = [PM_normalised*xi/2 for xi in XS]
K_can_avoid, _ = S.strong_approximation(K_can, avoid)
K_can_list = S.divisor_to_divisor_list(K_can_avoid)
AJ_bp_can = S.abel_jacobi(K_can_list)

# Let's set the basepoin to be a, and normalise
a_avoid, _ = S.strong_approximation(1*a, avoid)
a_list = S.divisor_to_divisor_list(a_avoid)
AJ_bp_a = S.abel_jacobi(a_list)
AJ_a_can = AJ_bp_can - 2*(g-1)*AJ_bp_a
AJ_a_can_normalised = AInv*AJ_a_can

# Use these to create a list of candidate RCVs using the fact that 2K == A(can). 
RCV_a0 = AJ_a_can_normalised/2
RCV_a_list = [RCV_a0+hl for hl in half_lattices_normalised]

# Create the theta function
from riemann_theta.riemann_theta import RiemannTheta
Tau = AInv*PM[:,g:]
RT = RiemannTheta(Tau)

# We evaluate the theta function at the RCV (with the correct sign for convention) for all candidates.
# We then record the index of the value which gives the smallest value, as this will be the correct one. 
# Note we need to set normaliesd=True as this is the condition required on the homology basis. 
Thetas = [RT(S.reduce_over_period_lattice(-Ki, normalised=True)).norm() for Ki in RCV_a_list]
mT = min(Thetas)
# make a check of accuracy:
if mT>1e-10:
    raise ValueError("A root of theta unlikely to be found")
ind = Thetas.index(mT)
RCV_a = RCV_a_list[ind]

# We now want to loop over homology transforms
# we copy the code from the period matrix notebook, with fewer comments
####
from sage.geometry.hyperbolic_space.hyperbolic_model import moebius_transform
j0 = -29^3*5*2^(-5)
alp = S._CCz.gen()
roots = (4*j0*alp*(1-alp)-1728).roots(multiplicities=False)
t1, t2 = (I*hypergeometric([1/6,5/6],[1],1-al).hypergeometric_simplify()/hypergeometric([1/6,5/6],[1],al).hypergeometric_simplify() 
          for al in roots)
if (t1.real_part()-1/2).abs()>1e-10:
    print("Swapping roots")
    t1 = t2
if (t1.real_part()-1/2).abs()>1e-10:
    raise ValueError("Invalid taus found, check calculation.")
t0 = moebius_transform(matrix(2,2,[-1,1,-2,1]), S._CC(t1)) - 1
if ((elliptic_j(t0)+29^3*5/2^5).norm()+(elliptic_j(5*t0)+25/2).norm())>1e-10:
    raise ValueError("Incorrect tau0 found")
RM_Braden = t0*matrix([[4,1,-1,1],[1,4,1,-1],[-1,1,4,1],[1,-1,1,4]])
Rs = integer_matrix_relations(Tau, RM_Braden)
r = len(Rs)
A = PolynomialRing(QQ, r, 'x')
gensA = A.gens()
R = sum(gensA[i] * Rs[i].change_ring(A) for i in range(r))
tr = (R * S.rosati_involution(R)).trace()
M = Matrix(ZZ, r, r, [tr.derivative(gen1).derivative(gen2)
                        for gen1 in gensA for gen2 in gensA])
vs = M.__pari__().qfminim(4*g)[2].sage().transpose()
vs = [v for v in vs if v * M * v == 4*g]
vs += [-v for v in vs]
RsIso = []
for v in vs:
    R = sum(v[i] * Rs[i] for i in range(r))
    if R * S.rosati_involution(R) == 1:
        RsIso.append(R)
if len(RsIso)!=240:
    raise ValueError("Not enough transforms found, error somewhere.")
####

# We recreate the function that reduces over the period lattice, now for an arbitrary complex torus
# of dimension g
def reduce_over_lattice(vector, Lambda):
    VR = VectorSpace(S._RR, 2*S.genus)
    VC = VectorSpace(S._CC, S.genus)
    I = S._CC(0,1)
    PM = Lambda        
    def C2R(v):
        return VR([z.real_part() for z in v]+[z.imag_part() for z in v])        
    u = C2R(vector)
    basis_vecs = [C2R(c) for c in PM.columns()]
    M = Matrix([[ei.dot_product(ej) for ei in basis_vecs] for ej in basis_vecs])
    v_dot_e = VR([u.dot_product(e) for e in basis_vecs])
    coeffs = M.solve_right(v_dot_e)
    u -= sum([t.round()*e for t, e in zip(coeffs, basis_vecs)])
    reduced = VC([S._CC(u[i]+I*u[i+S.genus]) for i in range(S.genus)])    
    return reduced

# Calculate the AJ image of the invariant theta characteristic and normalise 
Delta_avoid, _ = S.strong_approximation(Delta, avoid)
Delta_list = S.divisor_to_divisor_list(Delta_avoid)
AJ_bp_D = S.abel_jacobi(Delta_list)
AJ_a_D = AJ_bp_D - Delta.degree()*AJ_bp_a
AJ_a_D_normalised = AInv*AJ_a_D


# We now loop over the homology changes, checking the valid ones against Braden's result, 
# and then comparing AJ images
vers = []
for R0 in RsIso:
    D0 = R0[0:g, 0:g]
    B0 = R0[0:g, g:]
    C0 = R0[g:, 0:g]
    A0 = R0[g:, g:]
    if (max([el.norm() for el in (B0+Tau*A0-(D0+Tau*C0)*RM_Braden).list()])>1e-10):
        raise ValueError("Homoloy transform is invalid")
    mat1 = numerical_inverse(C0.transpose()*Tau+D0.transpose())
    mat2 = A0.transpose()*Tau+B0.transpose()
    char1 = vector([(C0.transpose()*D0)[j,j] for j in range(g)])/2
    char2 = vector([(A0.transpose()*B0)[j,j] for j in range(g)])/2
    RCV_a_RR = RCV_a*mat1 - char1*mat2*mat1-char2

    # We check that this still satisfies the theta condition.
    RTRR = RiemannTheta(RM_Braden)
    if RTRR(-RCV_a_RR).norm()>1e-10:
        raise ValueError("Error in transform of RCV")
    vers.append(reduce_over_lattice(vector([-12,-3,3,-3])/5+t0*vector([-6,-6,3,0])-2*RCV_a_RR, 
                          block_matrix(1,2,[Ig, RM_Braden])).norm())
    
for i in range(len(RsIso)):
    if vers[i]>1e-10:
        continue
    R0 = RsIso[i]
    D0 = R0[0:g, 0:g]
    B0 = R0[0:g, g:]
    C0 = R0[g:, 0:g]
    A0 = R0[g:, g:]
    mat1 = numerical_inverse(C0.transpose()*Tau+D0.transpose())
    mat2 = A0.transpose()*Tau+B0.transpose()
    char1 = vector([(C0.transpose()*D0)[j,j] for j in range(g)])/2
    char2 = vector([(A0.transpose()*B0)[j,j] for j in range(g)])/2
    RCV_a_RR = RCV_a*mat1 - char1*mat2*mat1-char2

    # We check thath this still satisfies the theta condition.
    RTRR = RiemannTheta(RM_Braden)
    if RTRR(-RCV_a_RR).norm()>1e-10:
        raise ValueError("Error in transform of RCV")
        
    t3 = (reduce_over_lattice(vector([3,2,-2,-3])/10+t0.imag_part()*I*vector([1,-2,-2,1])-RCV_a_RR, 
                          block_matrix(1,2,[Ig, RM_Braden])).norm())
    tt = (reduce_over_lattice(10*RCV_a_RR, block_matrix(1,2,[Ig, RM_Braden])).norm())
    
    tl = (reduce_over_lattice(AJ_a_D_normalised*mat1-RCV_a_RR, block_matrix(1,2,[Ig, RM_Braden])).norm())
    
    print(t3)
    print(tt)
    print(tl)
    break

88
3.1549214521349751490884859190359308563535312880296299127597e-59
3.4347492973019314207613351348925127583523870231732738650281e-58
6.8078496170392317663283084369977150723717808766489513508636e-24
